# Import libraries

In [1]:
# drive access
from google.colab import drive
drive.mount('/content/drive')

# standard library
import numpy as np
import pandas as pd

# for audio
import librosa
from IPython.display import Audio

# for file streaming
import os

# for model
!pip install transformers
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.3 MB/s eta 0:00:00


In [2]:
# load model and tokenizer
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Load examples from jmk

In [3]:
audio, _ = librosa.load('/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_jmk_arctic/wav/arctic_b0539.wav', sr=16000)

In [4]:
Audio(audio, rate=16000)

In [5]:
audio, _ = librosa.load('/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_jmk_arctic/wav/arctic_b0538.wav', sr=16000)

In [6]:
Audio(audio, rate=16000)

# get the files from bea_Amused folder & create auto transcription using wav2vec

In [7]:
audio_filename = os.listdir('/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_jmk_arctic/wav')

audio_filename

['arctic_a0529.wav',
 'arctic_a0281.wav',
 'arctic_a0096.wav',
 'arctic_a0114.wav',
 'arctic_a0116.wav',
 'arctic_a0464.wav',
 'arctic_a0556.wav',
 'arctic_a0549.wav',
 'arctic_a0444.wav',
 'arctic_b0059.wav',
 'arctic_a0502.wav',
 'arctic_a0245.wav',
 'arctic_a0231.wav',
 'arctic_b0087.wav',
 'arctic_a0123.wav',
 'arctic_b0152.wav',
 'arctic_a0191.wav',
 'arctic_a0235.wav',
 'arctic_a0338.wav',
 'arctic_b0137.wav',
 'arctic_a0453.wav',
 'arctic_b0045.wav',
 'arctic_a0368.wav',
 'arctic_a0555.wav',
 'arctic_a0543.wav',
 'arctic_a0422.wav',
 'arctic_b0074.wav',
 'arctic_b0199.wav',
 'arctic_a0204.wav',
 'arctic_a0205.wav',
 'arctic_a0181.wav',
 'arctic_b0104.wav',
 'arctic_a0398.wav',
 'arctic_a0430.wav',
 'arctic_b0050.wav',
 'arctic_a0516.wav',
 'arctic_a0413.wav',
 'arctic_a0399.wav',
 'arctic_a0192.wav',
 'arctic_b0184.wav',
 'arctic_b0187.wav',
 'arctic_a0315.wav',
 'arctic_a0172.wav',
 'arctic_b0057.wav',
 'arctic_a0588.wav',
 'arctic_a0544.wav',
 'arctic_a0212.wav',
 'arctic_a053

In [8]:
audio_df = pd.DataFrame()

audio_df['filename'] = audio_filename

audio_df.head()

,filename
0,arctic_a0529.wav
1,arctic_a0281.wav
2,arctic_a0096.wav
3,arctic_a0114.wav
4,arctic_a0116.wav


In [9]:
auto_transcription = []

for filename in audio_df['filename']:
  audio, _ = librosa.load(f'/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_jmk_arctic/wav/{filename}', sr=16000)
  input_value = processor(audio, return_tensors="pt", padding="longest", sampling_rate=16000).input_values
  predicted_id = torch.argmax(model(input_value).logits, dim=-1)
  transcription = processor.batch_decode(predicted_id)
  auto_transcription.append(transcription[0])
  del audio, input_value, predicted_id, transcription

auto_transcription

['AS I SAY HE HAD TAPPED THE MESSAGE VERY RAPIDLY',
 'I DO NOT BLAME YOU FOR ANYTHING REMEMBER THAT',
 'PERHAPS SHE HAD ALREADY MET HER FADE A LITTLE DEEPER IN THE FOREST',
 'HER WORDS SENT A STRANGE CHILL THROUGH FILIP',
 'WAS IT THE RENDEZVOUS OF THOSE WHO WERE STRIVING TO WORK HIS RUIN',
 'BETWEEN THE RUSH OF THE CASCADES STREAKS OF RUST SHOWED EVERYWHERE',
 "WHY HE'S BOUGHT FORTY POUNDS OF GOODS FROM YOU ALREADY",
 'DO YOU KNOW THAT WE WEIGH EVERY POUND OF CAL WE BURN',
 "THEN YOU DON'T BELIEVE IN ALTRAISM",
 'IN MOMENTS OF MENTAL ENERGY PHILIP WAS RESTLESS',
 'ANYTHING UNUSUAL OR ABNORMAL WAS SUFFICIENT TO SEND A FELLOW TO THE MOLOCAI',
 'OUT OF HIS EIGHTEEN HUNDRED HE LAY ASIDE SIXTEEN HUNDRED EACH YEAR',
 'THE RESIDENT COMMISSIONER IS AWAY IN AUSTRALIA',
 'THEY WILL SEARCH FOR US BETWEEN THEIR CAMP AND CHURCHILL',
 'SHE HAD DIED FROM COLD AND STARVATION',
 'OW A WILD DOG HE GROWLED',
 'FOR A TIME THE EXCITING THRILL OF HIS ADVENTURE WAS GONE',
 'HIS VOICE WAS PASSIONATELY REBELL

confirm all files from bea_amused have been auto transcribed & add to the df

In [10]:
assert len(audio_df) == len(auto_transcription)

In [11]:
audio_df['auto_transcription'] = auto_transcription

In [12]:
audio_df.head()

,filename,auto_transcription
0,arctic_a0529.wav,AS I SAY HE HAD TAPPED THE MESSAGE VERY RAPIDLY
1,arctic_a0281.wav,I DO NOT BLAME YOU FOR ANYTHING REMEMBER THAT
2,arctic_a0096.wav,PERHAPS SHE HAD ALREADY MET HER FADE A LITTLE ...
3,arctic_a0114.wav,HER WORDS SENT A STRANGE CHILL THROUGH FILIP
4,arctic_a0116.wav,WAS IT THE RENDEZVOUS OF THOSE WHO WERE STRIVI...


take a look at one example

In [13]:
example = audio_df['filename'].iloc[0]
audio, _ = librosa.load(f'/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_jmk_arctic/wav/{example}', sr=16000)

In [14]:
Audio(audio, rate=16000)

In [15]:
audio_df['auto_transcription'].iloc[0]

'AS I SAY HE HAD TAPPED THE MESSAGE VERY RAPIDLY'

save the df to csv file for future use

In [16]:
audio_df.to_csv('/content/drive/MyDrive/266/Data/Clean_Data/Arctic/auto_transcription/jmk.csv', index=False)

In [17]:
check = pd.read_csv('/content/drive/MyDrive/266/Data/Clean_Data/Arctic/auto_transcription/jmk.csv')

check.head()

,filename,auto_transcription
0,arctic_a0529.wav,AS I SAY HE HAD TAPPED THE MESSAGE VERY RAPIDLY
1,arctic_a0281.wav,I DO NOT BLAME YOU FOR ANYTHING REMEMBER THAT
2,arctic_a0096.wav,PERHAPS SHE HAD ALREADY MET HER FADE A LITTLE ...
3,arctic_a0114.wav,HER WORDS SENT A STRANGE CHILL THROUGH FILIP
4,arctic_a0116.wav,WAS IT THE RENDEZVOUS OF THOSE WHO WERE STRIVI...
